## Task 2 - Naive CUDA Kernel

In [16]:
%%writefile matrix_gpu_naive.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <sys/time.h>

__global__ void matmul(float *a, float *b, float *c, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < n && col < n) {
        float sum = 0.0f;
        for (int k = 0; k < n; k++) {
            sum += a[row * n + k] * b[k * n + col];
        }
        c[row * n + col] = sum;
    }
}

int main(int argc, char **argv) {
    int n = 512;
    if (argc > 1) {
        n = atoi(argv[1]);
    }

    printf("Matrix size: %d x %d\n", n, n);

    size_t size = n * n * sizeof(float);
    
    float *a = (float *)malloc(size);
    float *b = (float *)malloc(size);
    float *c = (float *)malloc(size);
    
    for (int i = 0; i < n * n; i++) {
        a[i] = rand() % 100 / 100.0f;
        b[i] = rand() % 100 / 100.0f;
    }
    
    float *d_a, *d_b, *d_c;
    
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
    
    dim3 threads(16, 16);
    dim3 blocks((n + 15) / 16, (n + 15) / 16);
    
    struct timeval t1, t2;
    
    gettimeofday(&t1, 0);
    matmul<<<blocks, threads>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();
    gettimeofday(&t2, 0);
    
    double time = (1000000.0 * (t2.tv_sec - t1.tv_sec) + t2.tv_usec - t1.tv_usec) / 1000.0;
    
    printf("Time: %f ms\n", time);
    
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);

    return 0;
}

Overwriting matrix_gpu_naive.cu


## Task 3 - Tiled CUDA code

In [22]:
!nvidia-smi

Sun Feb  1 03:29:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [17]:
!nvcc matrix_gpu_naive.cu -o matrix_gpu_naive

In [18]:
!./matrix_gpu_naive 512
!./matrix_gpu_naive 1024
!./matrix_gpu_naive 2048
!./matrix_gpu_naive 4096
!./matrix_gpu_naive 8192
!./matrix_gpu_naive 16384

Matrix size: 512 x 512
Time: 7.298000 ms
Matrix size: 1024 x 1024
Time: 7.330000 ms
Matrix size: 2048 x 2048
Time: 7.378000 ms
Matrix size: 4096 x 4096
Time: 7.217000 ms
Matrix size: 8192 x 8192
Time: 7.429000 ms
Matrix size: 16384 x 16384
Time: 7.377000 ms


## Task 4 - Optimizing CUDA Code

In [19]:
%%writefile matrix_gpu_tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <sys/time.h>
#define TILE 16

__global__ void matmul_tiled(float *a, float *b, float *c, int n) {
    __shared__ float tile_a[TILE][TILE];
    __shared__ float tile_b[TILE][TILE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int row = blockIdx.y * TILE + ty;
    int col = blockIdx.x * TILE + tx;

    float sum = 0.0f;

    for (int m = 0; m < (n + TILE - 1) / TILE; m++) {
        if (row < n && (m * TILE + tx) < n)
            tile_a[ty][tx] = a[row * n + m * TILE + tx];
        else
            tile_a[ty][tx] = 0.0f;

        if (col < n && (m * TILE + ty) < n)
            tile_b[ty][tx] = b[(m * TILE + ty) * n + col];
        else
            tile_b[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE; k++)
            sum += tile_a[ty][k] * tile_b[k][tx];

        __syncthreads();
    }

    if (row < n && col < n)
        c[row * n + col] = sum;
}

int main(int argc, char **argv) {
    int n = 512;
    if (argc > 1) {
        n = atoi(argv[1]);
    }

    printf("Matrix size: %d x %d\n", n, n);

    size_t size = n * n * sizeof(float);
    
    float *a = (float *)malloc(size);
    float *b = (float *)malloc(size);
    float *c = (float *)malloc(size);
    
    for (int i = 0; i < n * n; i++) {
        a[i] = rand() % 100 / 100.0f;
        b[i] = rand() % 100 / 100.0f;
    }
    
    float *d_a, *d_b, *d_c;
    
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
    
    dim3 threads(16, 16);
    dim3 blocks((n + 15) / 16, (n + 15) / 16);
    
    struct timeval t1, t2;
    
    gettimeofday(&t1, 0);
    matmul_tiled<<<blocks, threads>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();
    gettimeofday(&t2, 0);
    
    double time = (1000000.0 * (t2.tv_sec - t1.tv_sec) + t2.tv_usec - t1.tv_usec) / 1000.0;
    
    printf("Time: %f ms\n", time);
    
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);

    return 0;
}

Overwriting matrix_gpu_tiled.cu


In [37]:
!nvcc matrix_gpu_tiled.cu -o matrix_gpu_tiled

In [20]:
!./matrix_gpu_tiled 512
!./matrix_gpu_tiled 1024
!./matrix_gpu_tiled 2048
!./matrix_gpu_tiled 4096
!./matrix_gpu_tiled 8192
!./matrix_gpu_tiled 16384


Matrix size: 512 x 512
Time: 7.479584 ms
Matrix size: 1024 x 1024
Time: 7.357536 ms
Matrix size: 2048 x 2048
Time: 7.466752 ms
Matrix size: 4096 x 4096
Time: 7.754464 ms
Matrix size: 8192 x 8192
Time: 7.441408 ms
Matrix size: 16384 x 16384
Time: 7.374048 ms


## Task 5 - Performance Comparison

In [9]:
# data
python_512 = 7.81 * 1000
python_1024 = 65.46 * 1000
python_2048 = 839.49 * 1000

cpu_512  = 0.49 * 1000
cpu_1024 = 3.71 * 1000
cpu_2048 = 37.63 * 1000

naive_512  = 0.0125 * 1000
naive_1024 = 0.0109 * 1000
naive_2048 = 0.0107 * 1000

tiled_512  = 0.0088 * 1000
tiled_1024 = 0.0106 * 1000
tiled_2048 = 0.0109 * 1000


In [10]:
speedup_naive_512  = cpu_512  / naive_512
speedup_naive_1024 = cpu_1024 / naive_1024
speedup_naive_2048 = cpu_2048 / naive_2048

speedup_tiled_512  = cpu_512  / tiled_512
speedup_tiled_1024 = cpu_1024 / tiled_1024
speedup_tiled_2048 = cpu_2048 / tiled_2048

In [11]:
import pandas as pd

data = {
    "Implementation": [
        "CPU Time (ms)",
        "Naive CUDA Time (ms)",
        "Naive CUDA Speedup",
        "Tiled CUDA Time (ms)",
        "Tiled CUDA Speedup"
    ],
    "N = 512": [
        cpu_512,
        naive_512,
        speedup_naive_512,
        tiled_512,
        speedup_tiled_512
    ],
    "N = 1024": [
        cpu_1024,
        naive_1024,
        speedup_naive_1024,
        tiled_1024,
        speedup_tiled_1024
    ],
    "N = 2048": [
        cpu_2048,
        naive_2048,
        speedup_naive_2048,
        tiled_2048,
        speedup_tiled_2048
    ]
}

df = pd.DataFrame(data)
pd.options.display.float_format = "{:.2f}".format
df

,Implementation,N = 512,N = 1024,N = 2048
0,CPU Time (ms),490.00,3710.00,37630.00
1,Naive CUDA Time (ms),12.50,10.90,10.70
2,Naive CUDA Speedup,39.20,340.37,3516.82
3,Tiled CUDA Time (ms),8.80,10.60,10.90
4,Tiled CUDA Speedup,55.68,350.00,3452.29


## Step 6 - Using cuBLAS Library

In [22]:
%%writefile matrix_gpu_cublas.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main(int argc, char **argv) {
    int N;
    if (argc > 1) {
        N = atoi(argv[1]);
    } else {
        N = 1024; // default size
    }
    
    size_t size = N * N * sizeof(float);

    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() % 100 / 100.0f;
    }
    
    for (int i = 0; i < N * N; i++) {
        h_B[i] = rand() % 100 / 100.0f;
    }

    float *d_A;
    float *d_B;
    float *d_C;
    
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    cublasHandle_t handle;
    cublasCreate(&handle);

    float alpha = 1.0f;
    float beta  = 0.0f;

    cudaEvent_t start;
    cudaEvent_t stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    cublasSgemm(handle,
                CUBLAS_OP_N, CUBLAS_OP_N,
                N, N, N,
                &alpha,
                d_B, N,
                d_A, N,
                &beta,
                d_C, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("cuBLAS SGEMM time (N=%d): %f ms\n", N, ms);

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);   
    volatile float sink = h_C[0];

    cublasDestroy(handle);
    
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Overwriting matrix_gpu_cublas.cu


In [25]:
!nvcc matrix_gpu_cublas.cu -o matrix_gpu_cublas -lcublas

In [27]:
!./matrix_gpu_cublas 512
!./matrix_gpu_cublas 1024
!./matrix_gpu_cublas 2048

cuBLAS SGEMM time (N=512): 5.519136 ms
cuBLAS SGEMM time (N=1024): 6.127872 ms
cuBLAS SGEMM time (N=2048): 12.417696 ms


## Step 7 - Creating a Shared Library and Using it in Python

In [28]:
%%writefile matrix_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>
#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int Row = blockIdx.y * TILE_WIDTH + ty;
    int Col = blockIdx.x * TILE_WIDTH + tx;

    float Pvalue = 0.0f;

    int numTiles = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    for (int m = 0; m < numTiles; ++m) {
        int aCol = m * TILE_WIDTH + tx;
        if (Row < N && aCol < N) {
            ds_A[ty][tx] = A[Row * N + aCol];
        } else {
            ds_A[ty][tx] = 0.0f;
        }

        int bRow = m * TILE_WIDTH + ty;
        if (Col < N && bRow < N) {
            ds_B[ty][tx] = B[bRow * N + Col];
        } else {
            ds_B[ty][tx] = 0.0f;
        }

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k) {
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        }

        __syncthreads();
    }

    if (Row < N && Col < N) {
        C[Row * N + Col] = Pvalue;
    }
}

extern "C" void gpu_matrix_multiply(float *h_A, float *h_B, float *h_C, int N) {
    size_t size = N * N * sizeof(float);

    float *d_A;
    float *d_B;
    float *d_C;
    
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 block(TILE_WIDTH, TILE_WIDTH);
    
    int numBlocksX = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    int numBlocksY = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    dim3 grid(numBlocksX, numBlocksY);

    matrixMultiplyTiled<<<grid, block>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

Overwriting matrix_lib.cu


In [29]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu -o libmatrix.so

In [30]:
import ctypes
import numpy as np
import time
import sys
import os

In [31]:
lib = ctypes.cdll.LoadLibrary("./libmatrix.so")
lib.gpu_matrix_multiply.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int
]

N = 512

for i in range(5):
    print(f"\nTesting with matrix size N = {N}")
    
    A = np.random.rand(N, N).astype(np.float32)
    B = np.random.rand(N, N).astype(np.float32)
    C = np.zeros((N, N), dtype=np.float32)
    
    start = time.time()
    lib.gpu_matrix_multiply(A.ravel(), B.ravel(), C.ravel(), N)
    end = time.time()
    elapsed_time = end - start
    
    print(f"CUDA library time (N={N}): {elapsed_time * 1000:.4f} milliseconds")
    
    N = N * 2


Testing with matrix size N = 512
CUDA library time (N=512): 212.0955 milliseconds

Testing with matrix size N = 1024
CUDA library time (N=1024): 3.9580 milliseconds

Testing with matrix size N = 2048
CUDA library time (N=2048): 24.7159 milliseconds

Testing with matrix size N = 4096
CUDA library time (N=4096): 75.0914 milliseconds

Testing with matrix size N = 8192
CUDA library time (N=8192): 293.3767 milliseconds


## Step 8 - Adding Custom Functions to the Shared Library

In [32]:
%%writefile conv_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void convolution2D_GPU(
    unsigned char *image,
    int *kernel,
    int *output,
    int width,
    int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    int sum = 0;
    int k = 1;  // 3x3 kernel radius

    for (int ky = -k; ky <= k; ky++) {
        for (int kx = -k; kx <= k; kx++) {
            int ix = x + kx;
            int iy = y + ky;
            if (ix >= 0 && ix < width && iy >= 0 && iy < height) {
                int pixel = image[iy * width + ix];
                int weight = kernel[(ky + k) * 3 + (kx + k)];
                sum += pixel * weight;
            }
        }
    }
    output[y * width + x] = sum;
}

int main() {
    int width = 512;
    int height = 512;
    size_t img_size = width * height * sizeof(unsigned char);
    size_t out_size = width * height * sizeof(int);

    unsigned char *h_img = (unsigned char*)malloc(img_size);
    int *h_out = (int*)malloc(out_size);

    int h_kernel[9] = {
        -1, -1, -1,
        -1,  8, -1,
        -1, -1, -1
    };

    for (int i = 0; i < width * height; i++)
        h_img[i] = rand() % 256;

    unsigned char *d_img;
    int *d_kernel, *d_out;

    cudaMalloc(&d_img, img_size);
    cudaMalloc(&d_kernel, 9 * sizeof(int));
    cudaMalloc(&d_out, out_size);

    cudaMemcpy(d_img, h_img, img_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, 9 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 block(16, 16);
    dim3 grid((width + 15) / 16, (height + 15) / 16);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    convolution2D_GPU<<<grid, block>>>(d_img, d_kernel, d_out, width, height);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("CUDA convolution time: %f ms\n", ms);

    cudaMemcpy(h_out, d_out, out_size, cudaMemcpyDeviceToHost);

    cudaFree(d_img);
    cudaFree(d_kernel);
    cudaFree(d_out);
    free(h_img);
    free(h_out);

    return 0;
}


Overwriting conv_gpu.cu


In [33]:
!nvcc conv_gpu.cu -o conv_gpu

In [34]:
!./conv_gpu

CUDA convolution time: 7.379104 ms


#### Adding and Using the Convolution to your Custom Python Library

In [35]:
%%writefile conv_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void convolution2D_GPU(
    unsigned char *image,
    int *kernel,
    int *output,
    int width,
    int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    int sum = 0;
    int k = 1; // 3x3 kernel radius

    for (int ky = -k; ky <= k; ky++) {
        for (int kx = -k; kx <= k; kx++) {
            int ix = x + kx;
            int iy = y + ky;

            if (ix >= 0 && ix < width && iy >= 0 && iy < height) {
                int pixel = image[iy * width + ix];
                int weight = kernel[(ky + k) * 3 + (kx + k)];
                sum += pixel * weight;
            }
        }
    }

    output[y * width + x] = sum;
}

extern "C" void gpu_convolution(
    unsigned char *h_image,
    int *h_kernel,
    int *h_output,
    int width,
    int height
) {
    size_t img_size = width * height * sizeof(unsigned char);
    size_t out_size = width * height * sizeof(int);

    unsigned char *d_image;
    int *d_kernel;
    int *d_output;

    cudaMalloc((void**)&d_image, img_size);
    cudaMalloc((void**)&d_kernel, 9 * sizeof(int));
    cudaMalloc((void**)&d_output, out_size);

    cudaMemcpy(d_image, h_image, img_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, 9 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 block(16, 16);
    dim3 grid((width + 15) / 16, (height + 15) / 16);

    convolution2D_GPU<<<grid, block>>>(d_image, d_kernel, d_output, width, height);
    cudaDeviceSynchronize();

    cudaMemcpy(h_output, d_output, out_size, cudaMemcpyDeviceToHost);

    cudaFree(d_image);
    cudaFree(d_kernel);
    cudaFree(d_output);
}


Overwriting conv_lib.cu


In [36]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu conv_lib.cu -o libmatrix.so

In [37]:
!nm -D libmatrix.so | grep gpu_convolution

000000000000b0e5 T gpu_convolution


In [38]:
if './libmatrix.so' in sys.modules:
    del sys.modules['./libmatrix.so']

lib_path = os.path.abspath('./libmatrix.so')
lib = ctypes.CDLL(lib_path)

gpu_conv_func = lib.gpu_convolution
gpu_conv_func.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.int32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.int32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int,
    ctypes.c_int
]

width = 512
height = 512

image = np.random.randint(0, 256, size=(height, width), dtype=np.uint8)

kernel = np.array([
    -1, -1, -1,
    -1,  8, -1,
    -1, -1, -1
], dtype=np.int32)

output = np.zeros((height, width), dtype=np.int32)

start = time.time()
gpu_conv_func(
    image.ravel(),
    kernel,
    output.ravel(),
    width,
    height
)
end = time.time()

print(f"CUDA convolution time: {(end - start) * 1000:.4f} ms")

CUDA convolution time: 9.2390 ms
